In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("data_clean.csv")
df = df.drop(df.columns[0], axis = 1)
display(df.head())
df.shape

,date,year,gold,agnico,coal,copper,dax,dji,djt,eur,...,nasdaq,newmont,n_gas,oil,palladium,platinum,rgold,rio,silver,snp
0,2016-01-04,2016,1078.47,26.94,49.90,25.66,348.83,17148.94,7352.59,1.08,...,4903.09,18.44,2.33,36.76,544.55,881.65,37.18,28.41,13.98,2012.66
1,2016-01-05,2016,1079.33,27.62,50.25,25.95,350.98,17158.66,7363.95,1.07,...,4891.43,18.42,2.33,35.97,536.20,890.30,36.49,28.65,13.97,2016.71
2,2016-01-06,2016,1098.90,28.45,51.50,25.20,350.07,16906.51,7217.05,1.08,...,4835.76,18.56,2.27,33.97,505.60,879.55,37.02,27.03,14.07,1990.26
3,2016-01-07,2016,1103.45,29.99,51.25,24.56,341.24,16514.10,6995.39,1.09,...,4689.43,18.86,2.38,33.27,493.50,875.55,38.74,25.81,14.20,1943.09
4,2016-01-08,2016,1104.32,29.56,50.25,24.07,338.23,16346.45,6946.36,1.09,...,4643.63,18.01,2.47,33.16,494.20,880.00,37.59,25.07,13.94,1922.03


(1893, 21)

In [3]:
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### FUNCTION SHIFT PRICE

In [4]:
def shift_price(df, days_ahead):
    dftry = df.copy()
    dftry = df.reset_index(drop = True)
    dftry_gold = dftry[["date","gold"]].copy()
    dftry_ngold = dftry.drop("gold", axis = 1)
    hidden = dfty_ngold.iloc[-days_ahead:]
    dftry_ngold = dftry_ngold.shift(days_ahead, axis = 0)
    dftry = [dftry_gold, dftry_ngold]
    dftry = pd.concat(dftry, axis = 1)
    dftry = dftry.iloc[days_ahead:]
    new = dftry.iloc[-days_ahead:]
    data = dftry.iloc[0:-days_ahead]
    return new, data, hidden

### FUNCTION PREDICT

In [5]:
def predict(model, xtrain, xtest, xnew, ytrain, ytest, ynew, hidden):
    if model == "Linreg":
        regr = linear_model.LinearRegression()
        regr.fit(xtrain, ytrain)
        pred_train = regr.predict(xtrain)
        r2_train = r2_score(ytrain, pred_train)
        pred_test = regr.predict(xtest)
        r2_test = r2_score(ytest, pred_test)
        pred_new = regr.predict(xnew)
        r2_new = r2_score(ynew, pred_new)
        pred_hidden = regr.predict(hidden)
    else:
        if model == "mlp":
            regr = MLPRegressor(random_state = 1, max_iter = 500)
        elif model == "knn":
            regr = KNeighborsRegressor(n_neighbors=3, p = 1)
        elif model == "rforest":
            regr = RandomForestRegressor(max_depth = 2, random_state = 0)
        regr.fit(xtrain, ytrain)
        r2_train = regr.score(xtrain, ytrain)
        r2_test = regr.score(xtest, ytest)
        r2_new = regr.score(xnew, ynew)
        pred_hidden = regr.predict(hidden)
    pred_hidden = pd.DataFrame(pred_hidden, columns = [model])
    pred_hidden = round(pred_hidden,2)

    return r2_train, r2_test, r2_new, pred_hidden

In [22]:
def drop_corr(df):
    corr = df.corr()
    to_drop = corr.index[corr.gold < 0.2].tolist()
    return to_drop

In [6]:
def run_predictions(df,days_ahead, target):
    to_drop = drop_corr(df)
    new, data, hidden = shift_price(df, days_ahead)
    X = data.drop(target + to_drop, axis = 1)
    y = data[target].copy()
    X_new = new.drop(target + to_drop, axis = 1)
    y_new = new[target].copy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=61)
    X_hidden = hidden.drop(to_drop, axis = 1)
    num_train = X_train.select_dtypes(np.number)
    num_test = X_test.select_dtypes(np.number)
    num_new = X_new.select_dtypes(np.number)
    num_hidden =

    transformer = MinMaxScaler().fit(num_train)
    X_train_scaled = transformer.transform(num_train)
    X_test_scaled = transformer.transform(num_test)
    X_new_scaled = transformer.transform(num_new)
    
    a1,b1,c1,d1 = predict(X_train_scaled, X_test_scaled, X_new_scaled, y_train, y_test, y_new)

    return a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3,a4,b4,c4,d4,a5,b5,c5,d5